# Task 4

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import pandas as pd

# Load the updated dataframe
df = pd.read_csv('../Data/task_4_dataset.csv')

# Group by "State" and "County" and count the number of sightings
location_counts = df.groupby(['State', 'County']).size()

# Get the top 10 locations with the most sightings
top_10_locations = location_counts.nlargest(10).index

# Define a function to determine if a sighting is in a BigFoot Hotspot
def is_bigfoot_hotspot(row):
    return (row['State'], row['County']) in top_10_locations

# Apply the function to create the "BigFoot Hotspot" column
df['BigFoot Hotspot'] = df.apply(is_bigfoot_hotspot, axis=1)

# Save the updated dataframe
df.to_csv('../Data/updated_file.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# Mapping of state abbreviations to full names
state_mapping = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California",
    "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia",
    "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana", "IA": "Iowa",
    "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri",
    "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio",
    "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont",
    "VA": "Virginia", "WA": "Washington", "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}

def parse_submitted_date(df):
    df['Submitted Date'] = df['Submitted Date'].str.rstrip('.').str.strip()
    df['Year'] = pd.to_datetime(df['Submitted Date'], format='%A, %B %d, %Y', errors='coerce').dt.year
    df['Month'] = pd.to_datetime(df['Submitted Date'], format='%A, %B %d, %Y', errors='coerce').dt.month_name()
    df['Date'] = pd.to_datetime(df['Submitted Date'], format='%A, %B %d, %Y', errors='coerce').dt.day
    return df

def preprocess_df2(df):
    # Map state abbreviations to full names
    df['State'] = df['State'].map(state_mapping)
    # Drop duplicate rows based on 'Year', 'Month', 'Date', 'State', and 'County' (keep only the first occurrence)
    return df.drop_duplicates(subset=['Year', 'Month', 'Date', 'State', 'County'], keep='first')

def main():
    # Load the datasets
    df1 = pd.read_csv('../Data/updated_file.csv')
    df2 = pd.read_csv('../Data/updated_data.csv')

    # Use ThreadPoolExecutor to parallelize the parsing and preprocessing
    with ThreadPoolExecutor(max_workers=2) as executor:
        future_df1 = executor.submit(parse_submitted_date, df1)
        future_df2 = executor.submit(preprocess_df2, df2)

        # Wait for the futures to complete and get the results
        df1 = future_df1.result()
        df2_unique = future_df2.result()

    # Ensure 'Year', 'Month', 'Date' columns in df2 are in the correct format
    df2_unique['Year'] = pd.to_numeric(df2_unique['Year'], errors='coerce').fillna(-1).astype(int)
    df2_unique['Month'] = df2_unique['Month'].astype(str)
    df2_unique['Date'] = pd.to_numeric(df2_unique['Date'], errors='coerce').fillna(-1).astype(int)

    # Join the dataframes on 'Year', 'Month', 'Date', 'State', and 'County' using a left join
    result = pd.merge(df1, df2_unique, left_on=['Year', 'Month', 'Date', 'State', 'County'], right_on=['Year', 'Month', 'Date', 'State', 'County'], how='left')

    # Replace the placeholder (-1) with NaN where necessary
    result.replace(-1, np.nan, inplace=True)

    # Save the merged dataset to a new CSV file
    result.to_csv('task_4_dataset.csv', index=False)

if __name__ == '__main__':
    main()


<ipython-input-19-065ec2d77873>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_unique['Year'] = pd.to_numeric(df2_unique['Year'], errors='coerce').fillna(-1).astype(int)
<ipython-input-19-065ec2d77873>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_unique['Month'] = df2_unique['Month'].astype(str)
<ipython-input-19-065ec2d77873>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [ ]:
import pandas as pd
import datetime

# Assuming 'data.csv' is the input CSV file
input_csv = 'WeatherEvents_Jan2016-Dec2022.csv'
output_csv = 'updated_data.csv'  # The new file with additional columns

# Read the CSV data into a DataFrame.
df = pd.read_csv(input_csv, parse_dates=['StartTime(UTC)', 'EndTime(UTC)'])

# Extract 'Month', 'Year', and 'Date' from 'StartTime(UTC)' and add as new columns.
df['Month'] = df['StartTime(UTC)'].dt.strftime('%B')
df['Year'] = df['StartTime(UTC)'].dt.year
df['Date'] = df['StartTime(UTC)'].dt.day

# Save the updated dataframe to a new CSV file
df.to_csv(output_csv, index=False)

print(f"Updated CSV saved as {output_csv}")


Updated CSV saved as updated_data.csv


In [ ]:
!pip install openai




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from openai import OpenAI

# Set up the OpenAI client
client = OpenAI(
    api_key="key"
)


# Read the CSV file
df = pd.read_csv('../Data/task_4_dataset.csv')

# Function to estimate witness count using GPT-3.5
def estimate_witness_count(description):
    if not isinstance(description, str):
        return 0  # Return 0 if the description is not a string

    # Formulate the question
    question = f"How many people are mentioned here? Respond as just an integer: {description}"

    # Get the response from GPT-3.5
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": question}],
        model="gpt-3.5-turbo"
    )

    # Extract the estimated count from the response
    try:
        count = int(chat_completion.choices[0].message.content.strip())
    except ValueError:
        count = 0  # Default to 0 if the response is not a valid integer

    return count

# Apply the function to the "Other Witnesses" column
df['Witness Count'] = df['Other Witnesses'].apply(estimate_witness_count)

# Save the updated DataFrame to a new CSV file
df.to_csv('../Data/task_4_dataset.csv', index=False)

In [ ]:
import pandas as pd

# Load the updated dataframe
df = pd.read_csv('../Data/task_4_dataset.csv')

# Define a function to determine the "Multiple Witnesses" feature
def multiple_witnesses(row):
    if row['Witness Count'] > 2:
        if row['Class'] in ['Class A', 'Class B']:
            return True
        elif row['Class'] == 'Unknown':
            row['Class'] = 'Class B'
            return True
        elif row['Class'] == 'Class C':
            return True
    return False

# Apply the function to create the "Multiple Witnesses" column
df['Multiple Witnesses'] = df.apply(multiple_witnesses, axis=1)

# Save the updated dataframe
df.to_csv('../Data/task_4_dataset', index=False)


In [ ]:
import csv

# Input CSV file
input_csv_file = 'task_4_dataset.csv'

# Output TSV file
output_tsv_file = 'task_4_dataset.tsv'

# Read the CSV file and write to a TSV file
with open(input_csv_file, 'r') as csv_file, open(output_tsv_file, 'w', newline='') as tsv_file:
    csv_reader = csv.reader(csv_file)
    tsv_writer = csv.writer(tsv_file, delimiter='\t')

    for row in csv_reader:
        tsv_writer.writerow(row)

print(f'Converted {input_csv_file} to {output_tsv_file}')


Converted task_4_dataset.csv to task_4_dataset.tsv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('../Data/task_4_dataset.csv')

# Convert 'Witness Count' column to integer, skipping invalid literals
def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return None

df['Witness Count'] = df['Witness Count'].apply(safe_convert_to_int)

# Define a function to set the value of 'Multiple Witnesses' and 'Class' based on the conditions
def set_multiple_witnesses(row):
    if row['Witness Count'] is None:
        return None, None
    elif row['Witness Count'] < 2:
        return False, row['Class']
    elif row['Witness Count'] > 2 and (row['Class'] == 'Class A' or row['Class'] == 'Class B'):
        return True, row['Class']
    elif row['Class'] == 'Unknown':
        return True, 'Class B'
    elif row['Class'] == 'Class C':
        return True, row['Class']
    else:
        return False, row['Class']

# Apply the function to create the 'Multiple Witnesses' and 'Class' columns
df['Multiple Witnesses'], df['Class'] = zip(*df.apply(set_multiple_witnesses, axis=1))

# Save the updated dataset
df.to_csv('../Data/task_4_dataset', index=False)


# Task 5

In [ ]:
import pandas as pd

# Load the BFRO dataset
bfro_df = pd.read_csv('../Data/task_4_dataset.tsv', delimiter='\t')

# Load the environment dataset
environment_df = pd.read_csv('../Data/environment.tsv', delimiter='\t')

# Ensure the 'Year' column in BFRO is of the same data type for merging
bfro_df['Year'] = bfro_df['Year'].astype(str)

# For 'Year'-independent features, create a DataFrame indexed by 'State'
state_based_features = environment_df[['State', 'Forest cover rate', 'Number of lakes']].drop_duplicates(subset=['State'])

# Join the BFRO dataset with state-based features on 'State' to fill in these values
bfro_df = pd.merge(bfro_df, state_based_features, on='State', how='left')

# For the 'Year'-dependent feature ('Total Disasters'), we need to ensure it's correctly aligned by 'Year'
# Extract the 'Total Disasters' feature with 'Year' and 'State'
year_based_features = environment_df[['Year', 'State', 'Total Disasters']].copy()  # Use .copy() to explicitly create a copy

# Convert 'Year' in this DataFrame to string to match the BFRO 'Year' column
year_based_features['Year'] = year_based_features['Year'].astype(str)

# Merge this with the already enhanced BFRO DataFrame
bfro_df = pd.merge(bfro_df, year_based_features, on=['Year', 'State'], how='left')


In [ ]:
import requests

# Fetch and load the species distribution JSON data.
url = "https://data.ny.gov/api/views/tk82-7km5/rows.json?accessType=DOWNLOAD"
response = requests.get(url)
species_data = response.json()

# Create a DataFrame from the species distribution data.
species_df = pd.DataFrame.from_records(species_data['data'])

# Define the column headers as they appear in the JSON structure.
# This will depend on the structure of your JSON data.
column_headers = species_data['meta']['view']['columns']
columns = [col['name'] for col in column_headers]
species_df.columns = columns

# Define a function to safely parse the year and handle 'not available'
def parse_year(year_str):
    try:
        # If there's a range, take the latest year
        if isinstance(year_str, str) and '-' in year_str:
            return int(year_str.split('-')[1])
        # If it's a single year, return it as an integer
        return int(year_str)
    except ValueError:
        # If it's 'not available' or any other non-numeric value, return None or a default value
        return None

# Apply the function to the 'Year Last Documented' column
species_df['Year Last Documented Max'] = species_df['Year Last Documented'].apply(parse_year)

# Assuming species_df has been created and parsed correctly

# Group by 'County' to calculate the 'Wildlife Diversity'
wildlife_diversity = species_df.groupby('County').agg(Wildlife_Diversity=('Scientific Name', 'nunique')).reset_index()

# Define a function to check for significant conservation concern
def check_concern(row):
    concern_statuses = ['Endangered', 'Threatened']  # Define statuses that denote concern
    # Check if either NY or Federal listing status is in our concern_statuses
    if row['NY Listing Status'] in concern_statuses or row['Federal Listing Status'] in concern_statuses:
        return 1  # Significant conservation concern
    else:
        return 0  # No significant concern

# Apply the function to each row of the DataFrame
species_df['Conservation Concern'] = species_df.apply(check_concern, axis=1)

# Group by 'County' and find if there's any conservation concern within each county
conservation_concern = species_df.groupby('County')['Conservation Concern'].max().reset_index()

# Group by 'County' to find the most recent 'Year Last Documented'
last_documented_presence = species_df.groupby('County').agg(Last_Documented_Presence=('Year Last Documented Max', 'max')).reset_index()

# Merge the new features into the BFRO dataset based on the 'County' column.
bfro_df = bfro_df.merge(wildlife_diversity, on='County', how='left')
bfro_df = bfro_df.merge(conservation_concern, on='County', how='left')
bfro_df = bfro_df.merge(last_documented_presence, on='County', how='left')


In [ ]:
# Load the unemployment dataset
df_unemployment = pd.read_csv('../Data/unemployment.csv', delimiter=',')

# print(df_unemployment.columns)

# Filter columns to keep
columns_keep = ['Area_name', 'State', 'City/Suburb/Town/Rural',
                'Unemployment_rate_2000', 'Unemployment_rate_2001', 'Unemployment_rate_2002', 'Unemployment_rate_2003',
                'Unemployment_rate_2004', 'Unemployment_rate_2005', 'Unemployment_rate_2006', 'Unemployment_rate_2007',
                'Unemployment_rate_2008', 'Unemployment_rate_2009', 'Unemployment_rate_2010', 'Unemployment_rate_2011',
                'Unemployment_rate_2012', 'Unemployment_rate_2013', 'Unemployment_rate_2014', 'Unemployment_rate_2015',
                'Unemployment_rate_2016', 'Unemployment_rate_2017', 'Unemployment_rate_2018', 'Unemployment_rate_2019',
                'Unemployment_rate_2020', 'Median_Household_Income_2019']

# Filter columns to keep
columns_civilian = ['Area_name', 'State', 'City/Suburb/Town/Rural',
                    'Civilian_labor_force_2000', 'Civilian_labor_force_2001', 'Civilian_labor_force_2002',
                    'Civilian_labor_force_2003', 'Civilian_labor_force_2004', 'Civilian_labor_force_2005',
                    'Civilian_labor_force_2006', 'Civilian_labor_force_2007', 'Civilian_labor_force_2008',
                    'Civilian_labor_force_2009', 'Civilian_labor_force_2010', 'Civilian_labor_force_2011',
                    'Civilian_labor_force_2012', 'Civilian_labor_force_2013', 'Civilian_labor_force_2014',
                    'Civilian_labor_force_2015', 'Civilian_labor_force_2016', 'Civilian_labor_force_2017',
                    'Civilian_labor_force_2018', 'Civilian_labor_force_2019', 'Civilian_labor_force_2020',
                    'Median_Household_Income_2019']

# Make a new df with the columns to keep
df_filtered = df_unemployment[columns_keep].copy()
df_civilian = df_unemployment[columns_civilian].copy()

# Extract year from column names and create a new 'Year' column
years = df_filtered.columns.str.extract(r'(\d{4})').astype(float)  # Extract years as float
valid_years = years.dropna().astype(int)  # Drop NaN and convert to int
df_filtered['Year'] = valid_years  # Assign valid years to 'Year' column

# Extract year from column names and create a new 'Year' column
years = df_civilian.columns.str.extract(r'(\d{4})').astype(float)  # Extract years as float
valid_years = years.dropna().astype(int)  # Drop NaN and convert to int
df_civilian['Year'] = valid_years  # Assign valid years to 'Year' column

# Melt the dataframe to have one row per year per area
df_melted = df_filtered.melt(id_vars=['Area_name', 'State', 'City/Suburb/Town/Rural', 'Median_Household_Income_2019',
                                      'Year'],
                             value_vars=[col for col in df_filtered.columns if col.startswith('Unemployment_rate')],
                             var_name='Variable', value_name='Unemployment_rate')

df_civilian_melted = df_civilian.melt(id_vars=['Area_name', 'State', 'City/Suburb/Town/Rural',
                                               'Median_Household_Income_2019', 'Year'],
                                      value_vars=[col for col in df_civilian.columns if
                                                  col.startswith('Civilian_labor_force')],
                                      var_name='Variable', value_name='Civilian labor force')

# Extract year from 'Variable' column and assign it to 'Year' column
df_melted['Year'] = df_melted['Variable'].str.extract(r'(\d{4})').astype(int)

df_civilian_melted['Year'] = df_civilian_melted['Variable'].str.extract(r'(\d{4})').astype(int)

# Sort the dataframe by 'Area_name', 'State', and 'Year'
df_melted.sort_values(by=['Area_name', 'State', 'Year'], inplace=True)

df_civilian_melted.sort_values(by=['Area_name', 'State', 'Year'], inplace=True)

# Filter the DataFrame to drop rows where 'Area_name' contains only state names
df_melted = df_melted[~(df_melted['Area_name'].str.match(r'^[A-Za-z\s]+$') & ~df_melted['Area_name'].str.contains(','))]

df_civilian_melted = df_civilian_melted[~(df_civilian_melted['Area_name'].str.match(r'^[A-Za-z\s]+$') &
                                          ~df_civilian_melted['Area_name'].str.contains(','))]

# Split by comma and keep only the first part
df_melted['Area_name'] = df_melted['Area_name'].str.split(',').str[0].str.strip().str.strip('"')
df_melted['Area_name'] = df_melted['Area_name'].str.replace(r',\s*[A-Z]{2}', '')

df_civilian_melted['Area_name'] = df_civilian_melted['Area_name'].str.split(',').str[0].str.strip().str.strip('"')
df_civilian_melted['Area_name'] = df_civilian_melted['Area_name'].str.replace(r',\s*[A-Z]{2}', '')

# Create a function to remove the last word from a string
def remove_last_word(text):
    # Split the string by whitespace
    words = text.split()
    # Remove the last word
    modified_text = ' '.join(words[:-1])
    return modified_text

# Apply the function to the 'Area_name' column
df_melted['Area_name'] = df_melted['Area_name'].apply(remove_last_word)

df_civilian_melted['Area_name'] = df_civilian_melted['Area_name'].apply(remove_last_word)

# Display unique values in the 'State' column before mapping
# print("Unique state abbreviations before mapping:")
# print(df_melted['State'].unique())

# Map state abbreviations to full names
abbr_to_full = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'PR': 'Puerto Rico'
}

# Replace state abbreviations with full names using the mapping dictionary
df_melted['State'] = df_melted['State'].map(abbr_to_full)

df_civilian_melted['State'] = df_civilian_melted['State'].map(abbr_to_full)

# Display unique values in the 'State' column after mapping
# print("\nUnique state names after mapping:")
# print(df_melted['State'].unique())

# print(df_melted['Area_name'])
print("hello")

# Drop the 'Variable' column
df_melted.drop(columns=['Variable'], inplace=True)

df_civilian_melted.drop(columns=['Variable'], inplace=True)

# Rename the 'Area_name' column to 'County'
df_melted.rename(columns={'Area_name': 'County'}, inplace=True)

df_civilian_melted.rename(columns={'Area_name': 'County'}, inplace=True)

# Merge the two melted dataframes on 'County', 'State', 'City/Suburb/Town/Rural', and 'Year'
df_melted2 = df_melted.merge(df_civilian_melted,
                             on=['County', 'State', 'City/Suburb/Town/Rural', 'Year'],
                             how='outer')

# Drop the 'Median_Household_Income_2019_x' and 'Median_Household_Income_2019_y' columns
df_melted2.drop(columns=['Median_Household_Income_2019_x', 'Median_Household_Income_2019_y'], inplace=True)

# Save the merged DataFrame to a CSV file
# df_melted2.to_csv('merged_unemployment_civilian.csv', index=False)

# Replace "Southeast Fairbanks Census" with "Southeast Fairbanks" in the 'County' column
df_melted2['County'] = df_melted2['County'].replace('Southeast Fairbanks Census', 'Southeast Fairbanks')

df_melted2['County'] = df_melted2['County'].replace('Prince of Wales-Hyder Census', 'Prince of Wales')

df_melted2['County'] = df_melted2['County'].replace('Fairbanks North Star', 'Fairbanks')

df_melted2['County'] = df_melted2['County'].replace('Etowah', 'Etowa')

# Save the DataFrame to a CSV file
df_melted2.to_csv('merged_unemployment_civilian.csv', index=False)

# Convert 'Year' column in df_melted2 to string to match the 'Year' column in bfro_df
df_melted2['Year'] = df_melted2['Year'].astype(str)

# Perform left merge on 'County', 'Year', and 'State'
df_merged = bfro_df.merge(df_melted2, on=['County', 'Year', 'State'], how='left')

# Check the merged DataFrame
print(df_merged)

# Save df to a csv file
df_merged.to_csv('merged_bfro.csv', index=False)
# convert to a tsv file
df_merged.to_csv('merged_bfro.tsv', sep='\t', index=False)


hello
        Report Type     Id    Class                Submitted Date  \
0            Report  13038  Class A   Saturday, November 12, 2005   
1            Report   8792  Class B        Thursday, June 3, 2004   
2            Report   1255  Class B      Tuesday, October 5, 1999   
3            Report  11616  Class B           Friday, May 6, 2005   
4            Report    637  Class A     Monday, November 13, 2000   
...             ...    ...      ...                           ...   
5495  Media Article    519      NaN  Wednesday, November 15, 2006   
5496  Media Article    659      NaN      Monday, January 14, 2013   
5497  Media Article    733      NaN        Tuesday, March 4, 1919   
5498  Media Article    678      NaN    Wednesday, January 1, 2014   
5499  Media Article     59      NaN      Wednesday, June 18, 1980   

                                               Headline  Year  Season  \
0     Snowmobiler has encounter in deep snow near Po...  2005  Winter   
1     Four nocturna